## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [6]:
import glob

import numpy as np
import scipy as sp
import scipy.signal
import scipy.io
from matplotlib import pyplot as plt

%matplotlib osx 

#Helper function I created to plot traces for exploratory data analysis
# def plot(data_fl, fs):
#     """
#     Helper function created to plot traces

#     Plots PPG and accelerometer traces

#     Args:
#         data_fl: A data file containing trace data from the data_fls list
#         fs: sampling frequency in Hz
#     Returns:
#         Nothing
#     """
#     ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
#     ts = np.arange(len(ppg)) / fs
#     plt.clf()
#     plt.plot(ts, accx, label='x')
#     plt.plot(ts, accy, label='y')
#     plt.plot(ts, accz, label='z')
#     plt.plot(ts, ppg, label='ppg')
#     plt.title('PPG, ACCX, ACCY, ACCZ data plotted')
#     plt.legend()
#     plt.ylim((-50, 50))
#     plt.draw()
#     while not plt.waitforbuttonpress():
#         pass

#Helper function I created to plot spectrograms for exploratory data analysis
# def plot_spect(trace, fs):
#     """
#     Helper function to plot spectrograms

#     Plots spectrograms 

#     Args:
#         trace: either a ppg, accx, accy, or accz trace returned from the LoadTroikaDataFile(data_fl)
#         function
#         fs: sampling frequency in Hz
#     Returns:
#         Nothing
#     """
#     #Signal period in seconds 
#     T = len(trace)/fs/60
#     plt.figure()
#     plt.title('Spectrogram')
#     plt.specgram(trace, Fs=fs, NFFT=250, noverlap=125, xextent=((0, T)))
#     plt.xlabel('Time (sec)')
#     plt.ylabel('Frequency (Hz)')

#Helper function I created to plot ffts for exploratory data analysis
# def plot_fft(sig, fs):
#     """
#     Helper function to plot fourier transforms for signals

#     Plots fourier transforms

#     Args:
#         sig: either a ppg, accx, accy, or accz trace returned from the LoadTroikaDataFile(data_fl)
#         function
#     Returns:
#         Nothing
#     """
#     freqs = np.fft.rfftfreq(len(sig), 1/fs)
#     amps = np.fft.rfft(sig)
#     plt.figure(figsize=(12, 8))
#     plt.plot(freqs,np.abs(amps))
#     plt.xlabel('Frequency(Hz)')
#     plt.ylabel('Magnitude')


#Starter Code
def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def LoadTroikaRefFile(ref_fl):
    ref = sp.io.loadmat(ref_fl)
    return ref

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

#--------------------Algorithm functions start here and are below-------------------------#
    """
    Pulse rate estimatation algorithm
        This algorithm calculates an estimated pulse rate from raw photoplethysmogram(PPG) data in the presence of noise motion in the x,y,            and z axis from an accelerometer. The PPG and accelerometer data used with this algorithm was from the Troika dataset, and consists            of PPG and accelerometer data from 12 subjects wearing a wrist device while walking, running and during a brief cooldown on a                  treadmill. The reference labels for the data represents the subjects' pulse rate in beats per minute(bpm) and was calculated in eight          second windows, incremented every 2 seconds. The algorithm attempts to mimic this calculation, by first bandpass filtering the PPG and
        aggregated accelerometer signals, and then calculating the estimated pulse rate by computing the fast fourier transform(fft) for the           filtered PPG and accelerometer signals. After which, the frequency where the highest magnitude PPG signal exists is determined. This           frequency is multiplied by 60 seconds to obtain the estimated pulse rate. Next the estimated confidence that the frequency used to             determine the estimated pulse rate is calculated. This is done by dividing the spectral energy at that frequency by the total spectral         energy in the spectrum. The above process is performed in eight second windows on the PPG signal data, these                                   windows are incremented every 2 seconds. The algorithm also calculates the mean absolute error for each pulse rate estimate by                 first interpolating the estimated pulse rate to the reference label and then computing the mean absolute error between this                    interpolated pulse rate and the reference pulse rate. The algorithm returns a tuple of two numpy arrays, one numpy array is for                the mean absolute errors and the other is for the confidence estimates.

        The functions that define the core of the algorithm are below and are as follows:
            band_pass_filter                    - a function that returns a signal that has been band passed filtered
            moving_pulse_confidence_estimate    - a function to determine the pulse rate and confidence estimate in a given 8 second window                                                      incrementing every 2 seconds
            pulse_confidence_calc               - a function to calculate both the estimated pulse rate and confidence
            calc_errors                         - a function to calculate both mean absolute error between the estimated pulse rate and the                                                      reference label from the Troika dataset

            RunPulseRateAlgorithm               - From the starter code, this function calls the band_pass_filter and                                                                            moving_pulse_confidence_estimate to return mean absolute errors between the estimateed pulse
                                                  rate and the reference labels from the dataset along with confidence estimates for the
                                                  estimated pulse rate


    Returns:
        errors: (np.array): numpy array of mean absolute errors between the estimateed pulse rate
        confidences: (np.array): confidence estimates for the estimated pulse rate
    """
def band_pass_filter(signal, pass_band, fs):
    """Bandpass Filter.
    
    Args:
        signal: (np.array) The input signal
        pass_band: (tuple) The pass band. Frequency components outside 
            the two elements in the tuple will be removed.
        fs: (number) The sampling rate of <signal>
        
    Returns:
        (np.array) The filtered signal
    """
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def moving_pulse_confidence_estimate(ppg, acc, ref, fs):
    """a function to determine the pulse rate and confidence estimate in a given 8 second window incrementing every
    2 seconds, window and increment values were arrived at from the Troika dataset Readme.pdf
    
    Args:
        ppg: (np.array) The bandpass filtered ppg signal
        acc: (np.array) The bandpass filtered summed acc signal
        fs: (number) The sampling rate of <signal>
        ref: (np.array) a numpy array of reference pulse rates from the Troika dataset
        
    Returns:
        (tuple of numpy arrays) :  a tuple of numpy arrays for the mean absolute error between the ref label and estimated pulse rate in the            given window and the confidence estimate in the given window  
    """

    start_ind = 0
    # 8 second sampling window
    window = int(fs*8)
    end_ind = window
    # increment/output every 2 seconds 
    increment = int(fs*2)
    pr_ests = []
    confs = []
    while end_ind < len(ppg):
        ppg_w = ppg[start_ind:end_ind]
        acc_w = acc[start_ind:end_ind]

        pr_estimate, conf = pulse_confidence_calc(ppg_w, ppg, acc_w, fs)
        pr_ests.append(pr_estimate)
        confs.append(conf)

        start_ind += increment
        end_ind += increment
    errors = calc_errors(pr_ests, ref)
    return np.asarray(errors), np.asarray(confs)

def pulse_confidence_calc(ppg, full_ppg, acc, fs):
    """a function to calculate both the estimated pulse rate and confidence through the following method:
    1) The estimated pulse rate is calculated by taking the fast fourier transform(fft) of the ppg and acc signals
    in the given signal window. Then the frequency where the maximum fft magnitude for the ppg signal in relation to the accelerometer noide       is determined and multiplied by 60 seconds to determine the estimated pulse rate in beats per minute (bpm) for the window.

    2) The confidence is determined by summing the spectral energy at the frequency for the estimated pulse rate and dividing it by       the      spectral energy in the entire signal spectrum
    
    Args:
        ppg: (np.array) The bandpass filtered ppg signal for the given window
        full_ppg: (np.array) The full bandpass filtered ppg signal for confidence estimate calculation
        acc: (np.array) The bandpass filtered summed acc signal for the given window
        fs: (number) The sampling rate of <signal>
        
    Returns:
        pr_est, conf: (tuple - floating point numbers) : the pulse rate estimate in the given window, the confidence in the given window 
    """
    
    est_pr = 0
    conf_est = 0
    seconds = 60
    
    #compute fft freqs and mags for full bandpass filtered ppg signal
    full_ppg_freqs = np.fft.rfftfreq(len(full_ppg), 1 / fs)
    full_ppg_fft_mags = np.abs(np.fft.rfft(full_ppg))
    spectral_energy_ppg_spect = np.square(np.abs(full_ppg_fft_mags))

    #compute fft freqs, mags and freq where greatest ppg mag occurs
    ppg_freqs = np.fft.rfftfreq(len(ppg), 1 / fs)
    ppg_fft_mags = np.abs(np.fft.rfft(ppg))
    ppg_window = (ppg_freqs >= 0) & (ppg_freqs <= np.max(ppg_freqs))
    ppg_max_freq = ppg_freqs[np.argmax(ppg_fft_mags[ppg_window])]

    #compute fft freqs, mags and freq where greatest acc mag occurs
    acc_freqs = np.fft.rfftfreq(len(acc), 1 / fs)
    acc_fft_mags = np.abs(np.fft.rfft(acc))
    acc_window = (acc_freqs >= 0) & (acc_freqs <= np.max(acc_freqs))
    acc_max_freq = acc_freqs[np.argmax(acc_fft_mags[acc_window])]

    #if frequency where max ppg mag occurs is greater than or equal to
    #the frequency where the max acc mag occurs this is the estimated pulse rate
    #so compute it and the confidence estimate. 
    #In the case where the frequencies are equal assume that the magnitude for the ppg signal 
    #is much greater than the magnitude for the acc signal
    if ppg_max_freq >= acc_max_freq:
        est_pr = ppg_max_freq * seconds
        
    #if frequency where max ppg mag occurs is less than 
    #the frequency where the max acc mag occurs pick another frequency where the max
    #ppg mag occurs, this will be the estimated pulse rate,
    #compute it and the confidence estimate 
    elif acc_max_freq > ppg_max_freq:
        new_window = (ppg_freqs > acc_max_freq) | (ppg_freqs < acc_max_freq)
        ppg_max_freq = ppg_freqs[np.argmax(ppg_fft_mags[new_window])]
        est_pr = ppg_max_freq * seconds
    
    ppg_max_window = (ppg_freqs == ppg_max_freq)
    max_energy = ppg_fft_mags[ppg_max_window]
    spectral_energy_max_ppg = np.square(np.abs(max_energy))
    conf = np.sum(spectral_energy_max_ppg)/np.sum(spectral_energy_ppg_spect)

    return est_pr, conf
    
def calc_errors(estimated_pr, ref):
    """a function to calculate both mean absolute error between the estimated pulse rate and the reference label from the Troika dataset
    The function first interpolates the value of estimated pulse using the ref labels array then calculates the mean abolute error between         the estimated pulse rate and the reference label 
    
    Args:
        estimated_pr: (np.array): a numpy array of estimated pulse rates
        ref: (np.array) a numpy array of reference pulse rates from the Troika dataset
  
    Returns:
        maes:(list) : a list of element wise mean absolute errors between the estimated and ref pulse rates
    """
    ref_ts = ref[:]
    estimated_pr_ts = estimated_pr[:]
    est_pr_interp = np.interp(ref_ts, estimated_pr_ts, estimated_pr)

    maes = []
    for sig in zip(est_pr_interp ,ref):
        mae = np.mean(np.abs(sig[0]-sig[1]))
        maes.append(mae)
    return maes


def RunPulseRateAlgorithm(data_fl, ref_fl):
    """This is the main algorithm function, it loads a data file and reference label file from the Troika dataset, aggregates the accelerometer signals into a single signal, and then bandpass filters the PPG and aggregated accelerometer signals. These filtered signals, the reference label and sampling frequency are then inputted into the moving_pulse_confidence_estimate function. The output from the moving_pulse_confidence_estimate function which is a numpy array of mean abolute errors between the estimated heart rate and reference label and a numpy array of confidence estimates for the estimated pulse rates is returned from this function
    
    Args:
        data_fl: (str): a str of the filepath to the data file containing PPG and accelerometer data
        ref_fl: (str) a str of the filepath to the file reference pulse rate
  
    Returns:
        errors:(np.array) : a numpy array of mean absolute errors between the estimated pulse rates and the reference pulse rates.
        confidences: (np.array) : a numpy array of confidence estimates for the estimated pulse rate
    """
    # Load data and ref using LoadTroikaDataFile and LoadTroikaRefFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref = LoadTroikaRefFile(ref_fl)
    ref = ref['BPM0']

    # sum the accelerometer channels into one accelo channel to aggregate the motion artifact noise 
    sum_accelos = np.add(accx,accy,accz)

    #sampling frequency of 125Hz
    fs = 125

    # The passband was determined by looking at plots of the fft for the ppg and acc signals
    # and from the paper provided in the readme.md
    pass_band = [0.4, 5]
    f_ppg_signal = band_pass_filter(ppg, pass_band, fs)
    f_accelos_signal = band_pass_filter(sum_accelos, pass_band, fs)

    # Compute pulse rate estimates and estimation confidence.
    errors, confidences = moving_pulse_confidence_estimate(f_ppg_signal, f_accelos_signal, ref, fs)
      
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return errors, confidences

In [7]:
#EDA code
data_fls, ref_fls = LoadTroikaDataset()
ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[10])
fs = 125
ref = LoadTroikaRefFile(ref_fls[0])
ref = ref['BPM0']
# plot_spect(accz, fs)
# plot_fft(accz, fs)


-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

### Code Description
The algorithm is designed to calculate an estimated pulse rate from data collected from a photoplethysmogram(PPG) device, return an error between the estimated pulse rate and a reference pulse rate as well as a confidence regarding the estimated pulse rate. The algorithm can be run inside a python script or a jupyter notebook code block. It can be run by invoking the RunPulseRateAlgorithm function and supplying a data file and ref file from the Troika dataset.

### Data Description

The dataset used to develop this algorithm was obtained from the following publication:
        
         Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on     
         Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015.[Link](https:/ieeexplore.ieee.org/document/6905737)
         
The raw PPG and accelerometer data was stored in .mat files,  and consists of PPG and accelerometer data in the x,y and z axis from 12 subjects wearing a wrist device while walking, running and during a brief cooldown on a treadmill. The reference labels for the data represents the subjects' pulse rate in beats per minute(bpm)and was calculated in eight second windows, incremented every 2 seconds. The period from oscillating PPG generated by the PPG sensor and the signals generated by the accelerometer were used by the algorithm to estimate pulse rate.

Some of the shortcomings of the dataset could be the small number of subjects, at 12, yielding only 12 sets of PPG signals and labels, the absence of pulse rates taken from the subjects at rest without any motion, and the lack of any other general information on the subjects such as age and gender. 

The following plots were obtained from the PPG and accelerometer data to better understand the data
![](./figures/data0.png)

![](./figures/data0_zoomed.png)

![](./figures/fft_ppg_trace0_zoomed.png)

![](./figures/fft_accy_trace0_zoomed.png)

![](./figures/spect_ppg_trace0.png)

![](./figures/spect_accx_trace0.png)

### Algorithhm Description

#### How the algorithm works
In the main RunPulseRateAlgorithm function, first the data file containing the PPG and accelerometer data along with the file contained the reference pulse rates are loade. Then the PPG and aggregated accelerometer signals are bandpass filtered using the band_pass_filter function. These filtered signals are inputted, reference pulse rate and the sampling rate are inputted to the moving_pulse_confidence_estimate function. In the moving_pulse_confidence_estimate function the estimated pulse rate and confidence is calculated in eight second windows on the PPG signal data, these windows are incremented every 2 seconds. The estimated pulse rate and confidence is calculated in the pulse_confidence_calc function by computing the fast fourier transform(fft) for the filtered PPG and accelerometer signals. After which, the frequency where the highest magnitude PPG signal exists is determined. This frequency is multiplied by 60 seconds to obtain the estimated pulse rate. Next the estimated confidence that the frequency used to determine the estimated pulse rate is calculated. This is done by dividing the spectral energy at that frequency by the total spectral energy in the spectrum. In the calc_errs function which is called in the moving_pulse_confidence_estimate function, the algorithm calculates the mean absolute error for each pulse rate estimate by first interpolating the estimated pulse rate to the reference label and then computing the mean absolute error between this interpolated pulse rate and the reference pulse rate. The algorithm returns a tuple of two numpy arrays, one numpy array is for the mean absolute errors and the other is for the confidence estimates in the RunPulseRateAlgorithm function.


#### The specific aspects of the physiology that it takes advantage of

#### A description of the algorithm outputs

#### Caveats on algorithm outputs 

#### Common failure modes 
                     
### Algorithhm Performance






-----
### Next Steps
You will now go to **Test Your Algorithm** (back in the Project Classroom) to apply a unit test to confirm that your algorithm met the success criteria. 